# [07] Prior Box 만들기

객체 인식(Object Detection)에서 Object는 이미지 안에서 다양한 위치, 크기, 비율로 존재할 수 있습니다. 이러한 무한한 경우의 수에 대해서 확률이 절대 0이 아니지만, 모든 경우의 수를 고려하는 것은 계산상으로 불가능한 경우가 많습니다. 더군다나, 약간의 오차를 허용하는 것은 예측에 지장이 없기도 합니다.

따라서 `Prior Boxes`라는 개념이 등장합니다. 데이터 안에서 물체가 존재할만한 Bounding Box를 다양한 스케일로 미리 지정해 두는 것입니다.

In [3]:
from IPython.display import HTML, display

# Image from https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Object-Detection
display(HTML("<img src='img/[07]priors.png'>"))

---------------
### Prior Boxes

`Prior Boxes`는 Detection을 위해 사용되는 다양한 크기의 featuremap을 활용합니다. Prior들을 정하기 위해서는 다음의 3가지 요소가 활용됩니다.

- FeatureMap: 이전 실습에서 사용했던 `mid`, `end`, `features_a`, `features_b`, `features_c`가 여기에 사용됩니다.

- Scale : Feature map안에서 Object가 가지는 상대적인 크기를 의미합니다.
    - 같은 scale이라도 큰 feature map에서는 상대적으로 작은 면적을, 작은 feature map에서는 큰 면적을 차지합니다.
    
- Aspect Ratio : Prior Box의 Height:Width의 비율을 의미합니다.
    - 예를 들어, 1:1은 정사각형의 Box를 1:2는 직사각형의 Box를 의미합니다.
    - 숫자로 표시하였을 때, Ratio 2.0은 Height : Width = 1:2인 직사각형 Box입니다.


-------------------
## [Task 1] Create Prior Box
아래의 `create_prior_boxes`함수에는 우리가 만들고자 하는 prior box에 대한 feature map 크기, object의 scale, aspect ratio에 대한 정보가 설정되어 있습니다.

### ToDo: `create_prior_boxes` 함수 완성하기

주어진 값들을 바탕으로 prior box들을 만들어 봅시다.
- `prior_boxes` list에 [cx, cy, w, h]의 좌표값을 저장해야 합니다.
- cx, cy, w, h는 featuremap안에서의 상대적인 크기로 표현합니다. (0 ~ 1 값)
- featuremap마다, position마다, 주어진 scale대로, aspect ratio에 맞추어 box를 만들어 보세요.
- cx, cy의 좌표는 어떻게 구해야 할까요?
- w, h의 좌표는 어떻게 구해야 할까요?


In [4]:
import torch
from math import sqrt

def create_prior_boxes():
    # featuremap의 크기들을 지정합니다.
    feature_dims = {'mid': 18, 'end': 9, 'a': 9,
                 'b': 5, 'c': 3, 'd' : 1}
    
    # 각 featuremap에서 얻고지 하는 prior box의 크기를 결정합니다.
    obj_scales = {'mid': 0.2, 'end': 0.2, 'a': 0.375,
                 'b': 0.55, 'c': 0.725, 'd' : 0.9}
    
    # 각 featuremap에서 얻고자 하는 prior box의 모양을 결정합니다.
    aspect_ratio = {'mid': [1., 2., 3., 0.5, 0.333],
                    'end': [1., 2., 3., 0.5, 0.333], 
                    'a': [1., 2., 3., 0.5, 0.333],
                    'b': [1., 2., 3., 0.5, 0.333], 
                    'c': [1., 2., 0.5], 
                    'd' : [1., 2., 0.5]}
    
    # featuremap들의 이름을 저장합니다.
    fmaps = list(feature_dims.keys())
    
    prior_boxes = []
    
    # 모든 featuremap에 대해서
    for k, fmap in enumerate(fmaps):
        
        # featuremap의 각 i,j 위치에 대해서
        for i in range(feature_dims[fmap]):
            for j in range(feature_dims[fmap]):
                
                # [ToDo]: 해당 위치의 center_x, center_y의 좌표를 구합니다.
                cx = (? + ?) / feature_dims[fmap]
                cy = (? + ?) / feature_dims[fmap]
                
                # 얻고자 하는 prior box 모양들에 대해서
                for ratio in aspect_ratio[fmap]:
                    
                    # [ToDo]: prior box 좌표를 생성합니다. (cx, cy, w, h)
                    prior_coordinate = [??, ??, 
                                        ????????????, 
                                        ????????????]
                    
                    # prior box를 저장합니다.
                    prior_boxes.append(prior_coordinate)
    
    # prior box를 Tensor로 만들고, 0~1 사이의 값만을 가지도록 합니다.
    prior_boxes = torch.FloatTensor(prior_boxes)
    prior_boxes.clamp_(0, 1)

    return prior_boxes

In [5]:
prior_boxes = create_prior_boxes()

print('만들어야 하는 Prior Boxes Tensor Shape: [2585, 4]\n')
print('만들어진 Shape: {}\n'.format(prior_boxes.shape))

만들어야 하는 Prior Boxes Tensor Shape: [2585, 4]

만들어진 Shape: torch.Size([2585, 4])



-------------------
## [Task 2] Prior Box 추가하기
더 다양한 prior box를 가질 수 있도록 `현재 featuremap과 다음 featuremap의 기하평균`의 scale을 가지는 box를 추가해 보도록 하겠습니다.

### ToDo: Additional Prior Box
각 featuremap에 대해 scale이 sqrt(현재 fmap scale x 다음 fmap scale)이고 aspect ratio가 1인 prior box들을 추가해 봅시다.
주어진 값들을 바탕으로 prior box들을 만들어 봅시다.
- 마지막 featuremap ('c': 1x1)은 다음 featuremap이 없으므로 scale이 1인 box를 추가합니다.

In [6]:
import torch
from math import sqrt

def create_prior_boxes():
    feature_dims = {'mid': 18, 'end': 9, 'a': 9,
                 'b': 5, 'c': 3, 'd' : 1}

    obj_scales = {'mid': 0.2, 'end': 0.2, 'a': 0.375,
                 'b': 0.55, 'c': 0.725, 'd' : 0.9}

    aspect_ratio = {'mid': [1., 2., 3., 0.5, 0.333],
                    'end': [1., 2., 3., 0.5, 0.333], 
                    'a': [1., 2., 3., 0.5, 0.333],
                    'b': [1., 2., 3., 0.5, 0.333], 
                    'c': [1., 2., 0.5], 
                    'd' : [1., 2., 0.5]}

    fmaps = list(feature_dims.keys())

    prior_boxes = []

    for k, fmap in enumerate(fmaps):
        for i in range(feature_dims[fmap]):
            for j in range(feature_dims[fmap]):
                cx = (j + 0.5) / feature_dims[fmap]
                cy = (i + 0.5) / feature_dims[fmap]

                for ratio in aspect_ratio[fmap]:
                    prior_boxes.append([cx, cy, 
                                        obj_scales[fmap] * sqrt(ratio),
                                        obj_scales[fmap] / sqrt(ratio)])
                
                if fmap in ['mid',  'end', 'a', 'b', 'c']:
                    # [ToDo] : additional ratio를 계산합니다.
                    additional_scale = sqrt(obj_scales[????] * obj_scales[??????????])
                else:
                    additional_scale = 1.

                prior_boxes.append([cx, cy,
                                    additional_scale,
                                    additional_scale])

    prior_boxes = torch.FloatTensor(prior_boxes)
    prior_boxes.clamp_(0, 1)

    return prior_boxes

In [7]:
prior_boxes = create_prior_boxes()

print('만들어야 하는 Prior Boxes Tensor Shape: [3106, 4]\n')
print('만들어진 Shape: {}\n'.format(prior_boxes.shape))

만들어야 하는 Prior Boxes Tensor Shape: [3106, 4]

만들어진 Shape: torch.Size([3106, 4])



---------
### <생각해 봅시다>

- Prior Box의 개수가 많을 때와 적을 때의 장단점은 무엇인가요?
- Width:Height이 1:10의 비율을 가지는 등의 극단적인 형태의 Object는 어떻게 고려될까요?
------------